# Weights and Biases 101: Computer Vision trained on CIFAR10

## Setup

In [2]:
%pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.0/254.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb

In [ ]:
wandb.login()

## Utils

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def get_predictions(model, inputs, optimizer):
    optimizer.zero_grad()
    return model(inputs)

def update_model(model, data, loss_fn, optimizer):
    inputs, labels = data
    predictions = get_predictions(model, inputs, optimizer)
    loss = loss_fn(predictions, labels)
    loss.backward()
    optimizer.step()
    return loss.item()

def get_transforms(mean, std):
    return transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

def fetch_data(transform):
    return torchvision.datasets.CIFAR10(
        root='./data',
        train=True,
        download=True,
        transform=transform,
    )

def get_data_wo_transform():
    transform = transforms.Compose([transforms.ToTensor()])
    return fetch_data(transform)

def get_data(transforms, batch_size=4):
    trainset = fetch_data(transforms)
    return torch.utils.data.DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

def calculate_mean_std(dataset):
    dataloader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=64,
        shuffle=False,
        num_workers=2
    )

    channels_sum, channels_squared_sum, num_batches = 0, 0, 0

    for data, _ in dataloader:
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_squared_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

def save_model(model, path):
    torch.save(
        model.state_dict(),
        path
    )

## Run

In [ ]:
def train():
    config = {
        'batch_size': 2,
        'lr': .0002,
        'momentum': .9,
        'epochs': 2,
    }
    
    # create a W&B run w/ `wandb.init()`
    with wandb.init(project='wandb101', config=config):
        config = wandb.config
        
        # setup
        mean, std = calculate_mean_std(get_data_wo_transform())
        transforms = get_transforms(mean, std)
        data = get_data(transforms, config.batch_size)
        model = Net()
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.SGD(
            model.parameters(),
            lr=config.lr,
            momentum=config.momentum,
        )
        
        # training
        for epoch in range(config.epochs):
            for i, batch in enumerate(data):
                loss = update_model(
                    model,
                    batch,
                    loss_fn,
                    optimizer,
                )
                
                # store and log all results
                wandb.log({'epoch': epoch, 'loss': loss})
        path = './cifar_net.pth'
        save_model(model, path)
        
train()

In [32]:
# explicitly finish the run with W&B since the training run isn't running in this notebook
wandb.finish()